# Example of a hyperparameter sweep using Weights and Biases (wandb)

Often, there are a whole bunch of hyperparameters, such as learning rate, number of layers, width of an MLP, or batch size, that need to be chosen, without it being clear in advance what values will be good picks. 

A good way of finding good hyper parameter settings, is by just trying a bunch, and seeing what works best according to some pre-defined metric. 

Ideally, you should try to do this in an automated way. Using [wandb](https://wandb.ai/) helps with this. 

Additionally the `common_dl_utils` and `common_jax_utils` packages can help with setting such a wandb sweep up (or, if you prefer not to use wandb, they can help you automate things in different ways).

## In this notebook
In this notebook, we setup a hyperparameter sweep for the same model we saw in `inr_example.ipynb`. Basically, how this works is:
* We create a config detailing all hyperparameters (both fixed and varying)
* We start a wandb sweep using this config. This results in a sweep id.
* We run experiments using this sweep id. 

This last part can be done by calling `run_from_inr_sweep.py` with the commandline argument `--sweep_id=` followed by the correct id. When doing this locally, you can just do this on the command line. When using Snellius, you'll have to create a script for the job that loads the correct environment, details what resources are needed for how long, and calls `run_from_inr_sweep.py` with the correct sweep_id.

The way `run_from_inr_sweep.py` works is that it launches a wandb "agent" with a function for running the experiment. This "agent" receives a config from wandb with picks for the hyperparameters. From that point on, basically you have a config specifying a single run, and things work very similarly to what is done in `inr_example.ipynb` from the point you have your config there. 

## Type of sweep
Weights and Biases provides three options for doing these sweeps: you can either do a grid search, a random search, or a Bayesian search. Keep in mind that you have limited computational budget, so a grid search easily becomes unfeasible. 

In most cases, a random search will likely give you the best experience (and the least headaches). When running experiments from sweeps, you have the option to have a single agent perform multiple runs in sequence. However, somehow there seems to be a bug in wandb that causes the gpu memory to not always be freed up after each run, which can lead to OOM errors (both when using JAX and when using Pytorch). The only way I've found to reliably circumvent this, is to just keep `count` set to 1, and create a bunch of agents. 

However, when doing Bayesian search it seems that each agent still tries to create a new run after its first and single run, which just creates a lot of runs in your sweep that didn't really do anything. So my advice would be: don't waste your compute budget on grid search and don't waste your good mood on Bayesian search, unless you really need to.

### Grid search
If for some reason you really do need to do grid search, there might be better ways than doing this through a wandb sweep. The tools in `common_dl_utils.config_creation` allow for creating individual run configs (just like wandb does) in a way that some variables can be linked together. 

E.g. you want to vary the latent size, and you want the hidden size to always be twice the latent size. Or you want to vary what type of activation function you use for your INR, and you want to vary some hyper parameters, such as `w0` for Siren, but only if the corresponding layer is being used. 

In such cases, you might want to use the tools from `common_dl_utils.config_creation` to just create a folder full of config files for individual runs, and create a script that loops over those configs and runs the corresponding experiment (or ideally do something smarter than this so you can have a bunch of scripts do experiments in parallel on Snellius).

### Random seeds
If you want to vary only the prng seed (for random number generation) instead of any hyperparameters (or together with those hyper parameters) you can specify a 'prng_seed' in the config (this should be an integer).

If you don't specify a prng_seed in the config, the tools used for running the experiments will create a random prng seed and log it to wandb so as to make re-producing results easier.


In [ ]:

import wandb

import common_dl_utils as cdu
from common_dl_utils.config_creation import Config, VariableCollector

wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

wandb: Network error (ConnectionError), entering retry loop.


In [2]:
config = Config()
variable = VariableCollector()  # we'll use this to keep track of all varying hyperparameters
# when working with wandb, this is basically just syntactic sugar
# but if you want to do hyperparameter optimization without wandb, this can help set things up in other ways too.

# first we specify what the model should look like
config.architecture = './model_components'  # module containing all relevant classes for architectures
# NB if the classes relevant for creating the model are spread over multiple modules, this is no problem
# let config.architecture be the module that contains the "main" model class, and for all other components just specify the module
# or specify the other modules as default modules to the tools in common_jax_utils.run_utils
config.model_type = 'inr_modules.CombinedINR'

config.model_config = Config()
config.model_config.in_size = 2
config.model_config.out_size = 3
config.model_config.terms = [  
    # CombinedINR uses multiple MLPs and returns the sum of their outputs. These 'terms' are the MLPs
    ('inr_modules.MLPINR.from_config',{
        'hidden_size': 256,  # you can also specify the probability of each option
        'num_layers': 5,  # indicate the options that the hyperparameter sweep can choose from
        'layer_type': 'inr_layers.SirenLayer', 
        'num_splits': 1,
        'use_complex': False,
        'activation_kwargs': {'w0': variable(distribution="uniform", min=10., max=60.)}, # or specify a distribution from https://docs.wandb.ai/guides/sweeps/sweep-config-keys#distribution-options-for-random-and-bayesian-search
        'initialization_scheme':'initialization_schemes.siren_scheme',
        'positional_encoding_layer': ('inr_layers.ClassicalPositionalEncoding.from_config', {'num_frequencies': 10}),
    }),
    # ('inr_modules.MLPINR.from_config',{
    #     'hidden_size': 1024,  # you can also specify the probability of each option
    #     'num_layers': 2,
    #     'num_splits': 1,
    #     'layer_type': 'inr_layers.GaussianINRLayer',
    #     'use_complex': False,
    #     'activation_kwargs': {'inverse_scale': variable(distribution="uniform", min=1., max=10.)},
    # }),
    # ('inr_modules.MLPINR.from_config',{
    #     'hidden_size': 256,  # you can also specify the probability of each option
    #     'num_layers': 5,
    #     'layer_type': 'inr_layers.FinerLayer',
    #     'num_splits': 1,
    #     'use_complex': False,
    #     'activation_kwargs': {'w0': variable(distribution="uniform", min=2., max=40.)},#
    #     'initialization_scheme':'initialization_schemes.finer_scheme',
    #     'initialization_scheme_kwargs':{'bias_k': variable([-1,1], [-5,5], [-10,10], [-20,20], probabilities=[0.25, 0.25, 0.25, 0.25])},
    #     #'initialization_scheme_kwargs':{'scale_factor': variable(distribution="uniform", min=1., max=10.)}
    # }),
    # ('inr_modules.MLPINR.from_config',{
    #     'hidden_size': 1024,
    #     'num_layers': 2,
    #     'num_splits': 1,
    #     'layer_type': 'inr_layers.ComplexWIRE',
    #     'activation_kwargs': {'w0': variable(distribution="log_uniform_values", min=5, max=50), 's0': variable(distribution="uniform", min=0.1, max=10.)},
    #     'initialization_scheme':'initialization_schemes.siren_scheme',
    # }),
]

# variable(256, 512, 1024, probabilities=[0.33, 0.33, 0.33])
# variable(4, 5, 6)
# next, we set up the training loop, including the 'target_function' that we want to mimic
config.trainer_module = './inr_utils/'  # similarly to config.architecture above, here we just specify in what module to look for objects by default
config.trainer_type = 'training.train_inr'
config.target_function = 'images.ContinuousImage'
config.target_function_config = {
    'image': './example_data/parrot.png',
    'scale_to_01': True,
    'interpolation_method': 'images.make_piece_wise_constant_interpolation'
}
config.loss_function = 'losses.scaled_mse_loss'
config.sampler = ('sampling.GridSubsetSampler',{  # samples coordinates in a fixed grid, that should in this case coincide with the pixel locations in the image
    'size': [2040, 1356],
    'batch_size': 2000,
    'allow_duplicates': False,
})

config.optimizer = 'adam'  # we'll have to add optax to the additional default modules later
config.optimizer_config = {
    'learning_rate': variable(distribution="log_uniform_values", min=1e-6, max=1e-4)  # NB specifying a distribution only works for random search and bayesion search, not for grid search
}
config.steps = 40000
config.use_wandb = True

# now we want some extra things, like logging, to happen during training
# the inr_utils.training.train_inr function allows for this through callbacks.
# The callbacks we want to use can be found in inr_utils.callbacks
config.after_step_callback = 'callbacks.ComposedCallback'
config.after_step_callback_config = {
    'callbacks':[
        ('callbacks.print_loss', {'after_every':400}),  # only print the loss every 400th step
        'callbacks.report_loss',  # but log the loss to wandb after every step
        ('callbacks.MetricCollectingCallback', # this thing will help us collect metrics and log images to wandb
             {'metric_collector':'metrics.MetricCollector'}
        ),
        'callbacks.raise_error_on_nan'  # stop training if the loss becomes NaN
    ],
    'show_logs': False
}

config.metric_collector_config = {  # the metrics for MetricCollectingCallback / metrics.MetricCollector
    'metrics':[
        ('metrics.PlotOnGrid2D', {'grid': 256, 'batch_size':8*256, 'frequency':'every_n_batches'}),  
        # ^ plots the image on this fixed grid so we can visually inspect the inr on wandb
        ('metrics.MSEOnFixedGrid', {'grid': [2040, 1356], 'batch_size':2040, 'frequency': 'every_n_batches'})
        # ^ compute the MSE with the actual image pixels
    ],
    'batch_frequency': 400,  # compute all of these metrics every 400 batches
    'epoch_frequency': 1  # not actually used
}

config.after_training_callback = None  # don't care for one now, but you could have this e.g. store some nice loss plots if you're not using wandb 
config.optimizer_state = None  # we're starting from scratch

Originally, wandb didn't really deal with nested configurations like the one above very well. I think nowadays, the situation might be better, but I don't care to give myself a headache by finding out what the exact caveats to that are. 

So instead we'll make a flat config out of the nested config above
and the `common_jax_utils.wandb_utils.run_from_wandb` function that we use to actually run the experiments, will unflatten it when needed. 

In [3]:

flat_parameter_config = cdu.config_creation.make_flat_config(config)
#pprint(flat_parameter_config)  # uncomment to see what the flattened config looks like

Finally, we setup a sweep config detailing the method of the sweep (random in this case), the metric that is to be tracked, and the above `flat_parameter_config`.

In [4]:
sweep_config = {
    'name': 'inr_example_sweep',
    'method': 'random',
    'metric': {'name':'MSE_on_fixed_grid', 'goal':'minimize'},
    'parameters': flat_parameter_config,
    'description': 'An example of a hyperparameter sweep for training an INR'
}

In [5]:
sweep_id = wandb.sweep(sweep_config, entity="bep-circle", project="inr_edu_24")
print(f"nohup python run_from_inr_sweep.py --sweep_id={sweep_id} > inr_sweep_example.out")

wandb: Network error (ConnectionError), entering retry loop.
403 response executing GraphQL.
{"errors":[{"message":"permission denied","path":["upsertSweep"],"extensions":{"code":"PERMISSION_ERROR"}}],"data":{"upsertSweep":null}}
wandb: ERROR Error while calling W&B API: permission denied (<Response [403]>)


UsageError: permission denied

Next up, perform a single run for this sweep locally by typing `nohup python run_from_inr_sweep.py --sweep_id=0t56ux9d > inr_sweep_example.out`

Or run it on Snellius by submitting a job script that logs into wandb and runs `run_from_inr_sweep.py` with the correct sweep_id.